# Standardised Approach for Counterparty Credit Risk

Abbreviations: <br >
- $VM$: Variation Margin
- $EAD$: Exposure At Default under SA
- $RC$: replacement cost (different for margined and unmargined)
- $PFE$: Potential Future Exposure
- $V$: Current Market Value of the Derivative in the netting set
- $C$: Net harcut value of the net collateral held
- $ICA$: Independent Collateral Amount
- $TH$: Positive threshold before the counterparty must send the bank collateral
- $MTA$: Minimum transfer amount
- $NICA$: Net independent Collateral Amount
- $MF_i^{type}$: Maturity Factor
- $M_i$: Maturity (i remaining maturity floored by 10 business days)
- $S_i$: Start
- $E_i$: end date
- $T_i$: Contractual Exercise Date
- ${SD}_i$: Supervisory Duration
- $\delta_i$: Supervisory delta Adjustments
- $MPOR_i$: Margin period of risk appropriate for the margin agreement containing the transaction i

Formulas: <br> 
- EAD = alpha * (RC + PFE) 
- RC (Margined) = max{V - C;TH + MTA - NICA; 0}
- PFE = multiplier * $AddOn^{aggregate}$
- multipiler = min{1; Floor + (1 - Floor) * exp ($\frac{V - C}{2* (1-Floor) * AddOn^{aggregate} }$)}
- $AddOn^{aggregate}$ = $\sum_{a} AddOn^a$
- ${SD}_i$ = $\frac {exp(-0.05*S_i)-exp(-0.05*Ei)} {0.05}$
- ${MF}_i^{unmargined}$ = $\sqrt{\frac {min\{M_i; 1 year\}}{1year}}$
- ${MF}_i^{(margined)}$ = $\frac {3} {2} \sqrt{\frac{MPOR_i}{1year}}$

<a href="https://www.bis.org/publ/bcbs279.pdf">PDF Reference</a>

In [1]:
# csv file sample input


<h3>Example 1: <h3/>

In [2]:
import pandas as pd
import math
import numpy as np

In [3]:
fileName = "example_1_input.csv"
example_1_df = pd.read_csv(fileName)

In [4]:
example_1_df.head

<bound method NDFrame.head of    Trade #              Nature Residual maturity Base currency  \
0        1  Interest rate swap          10 years           USD   
1        2  Interest rate swap           4 years           USD   
2        3   European swaption   1 into 10 years           EUR   

   Notional (thousands) Pay Leg (*) Receive Leg (*)  Market value (thousands)  
0                 10000       Fixed        Floating                        30  
1                 10000    Floating           Fixed                       -20  
2                  5000    Floating           Fixed                        50  >

In [5]:
#rc(df) takes in a dataframe and compute the RC of a given file
def rc(df):
    v_c = df[['Market value (thousands)']].sum()
    r_c = max(int(v_c), 0)
    return r_c

# convert residual maturity to two lists, S and E by taking in
# three variables, emptyEi, emptySi and residual maturity
def residual_maturity_converter(Ei, Si, residual_maturity):
    for item in residual_maturity:
        item = str(item)
        idx = item.find("into")
        if (idx == -1): # S would be 0
            Si.append(0)
            end = int(item[:item.find("year") - 1])
            Ei.append(end)
        else:
            idx -= 1
            start = int(item[:idx])
            Si.append(start)
            idx += 5
            end = int(item[idx:item.find("year") - 1])
            Ei.append(end)
            
        for i in range(0,len(Ei)):
            Ei[i] = Ei[i] + Si[i]
            
#SD_generator takes in Si and Ei and generate the appropriate SD as a list
def SD_generator(Si, Ei, SD):
    if (len(Si) != len(Ei)):
        return "Error"
    for i in range(0, len(Si)):
        si = Si[i]
        ei = Ei[i]
        sd = (math.exp(-0.05*si) - math.exp(-0.05*ei)) / 0.05
        SD.append(sd)

In [6]:
rc(example_1_df)

60

In [7]:
example_1_df.columns

Index(['Trade #', 'Nature', 'Residual maturity', 'Base currency',
       'Notional (thousands)', 'Pay Leg (*)', 'Receive Leg (*)',
       'Market value (thousands)'],
      dtype='object')

In [8]:
# Creating a new dataframe for further analysis
data = {}
hedging_set = example_1_df['Base currency'].tolist()
data.update({"Hedging set": hedging_set})
data

{'Hedging set': ['USD', 'USD', 'EUR']}

In [9]:
Ei = []
Si = []
residual_maturity_converter(Ei, Si, example_1_df['Residual maturity'])

In [10]:
Ei

[10, 4, 11]

In [11]:
Si

[0, 0, 1]

In [12]:
data.update({"Notional (thousands)": example_1_df['Notional (thousands)'].tolist(),
            "Si": Si, "Ei":Ei})
data

{'Hedging set': ['USD', 'USD', 'EUR'],
 'Notional (thousands)': [10000, 10000, 5000],
 'Si': [0, 0, 1],
 'Ei': [10, 4, 11]}

In [13]:
SD = []
SD_generator(Si, Ei, SD)
SD

[7.8693868057473315, 3.6253849384403636, 7.485592282404547]

In [14]:
Adj_notion = np.array(SD) * np.array(example_1_df['Notional (thousands)'].tolist())

In [15]:
data.update({"SDi": SD,
            "Adjusted notional (thousands)": Adj_notion.tolist()})
data

{'Hedging set': ['USD', 'USD', 'EUR'],
 'Notional (thousands)': [10000, 10000, 5000],
 'Si': [0, 0, 1],
 'Ei': [10, 4, 11],
 'SDi': [7.8693868057473315, 3.6253849384403636, 7.485592282404547],
 'Adjusted notional (thousands)': [78693.86805747332,
  36253.849384403635,
  37427.961412022734]}

In [16]:
middle_df = pd.DataFrame(data)
middle_df

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands)
0,USD,10000,0,10,7.869387,78693.868057
1,USD,10000,0,4,3.625385,36253.849384
2,EUR,5000,1,11,7.485592,37427.961412


In [17]:
(x,y) = example_1_df.shape
x

3

In [18]:
middle_df.at[0,'Ei']

10

In [19]:
# phi() takes in a number and calculate the phi of it
def phi(x):
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

# supervisory_delta_generator() takes in two dataframe, the original one
# and the new generated one to modify and add a column on the 
# new generated dataframe
def supervisory_delta_generator(original_df, middle_df):
    (row_num, col_num) = original_df.shape
    sd_column = []
    for i in range(0, row_num):
        sd = None
        
        fixed = (str(original_df.at[i, 'Pay Leg (*)']) == 'Fixed')
        option_idx = (str(original_df.at[i, 'Nature']).find("swaption"))
        
        if (fixed and option_idx == -1): # long but not option
            sd = 1
        elif (not fixed and option_idx == -1): # short but not option
            sd = -1
        else: # option
            symbol = 1
            if not fixed: #if it is call option
                symbol = -1
            Ti = int(middle_df.at[i, 'Si'])
            sd = symbol * phi(symbol * 
                              (math.log(0.06 / 0.05) + 
                              0.5 * 0.5 ** 2 * Ti) / 
                              (0.5 * math.sqrt(Ti)))
        
        sd_column.append(sd)
        
    middle_df['Supervisory delta'] = sd_column

In [20]:
supervisory_delta_generator(example_1_df, middle_df)

In [21]:
middle_df

,Hedging set,Notional (thousands),Si,Ei,SDi,Adjusted notional (thousands),Supervisory delta
0,USD,10000,0,10,7.869387,78693.868057,1.000000
1,USD,10000,0,4,3.625385,36253.849384,-1.000000
2,EUR,5000,1,11,7.485592,37427.961412,-0.269395


In [22]:
#time_bucket_classifer() classifies each transaction by time
def time_bucket_classifier(original_df, middle_df):
    classes = []
    for number in middle_df['Ei'].tolist():
        if number > 5:
            classes.append(3)
        elif number < 1:
            classes.append(1)
        else:
            classes.append(2)
    middle_df['Time Bucket'] = classes

    
# Hedging list generator
def hedging_list_generator(middle_df):
    series1 = middle_df['Adjusted notional (thousands)']
    series2 = middle_df['Supervisory delta']
    hedging_set = series1 * series2
    return hedging_set.tolist()

In [23]:
middle_df['Supervisory delta']

0    1.000000
1   -1.000000
2   -0.269395
Name: Supervisory delta, dtype: float64

In [24]:
middle_df['Adjusted notional (thousands)']

0    78693.868057
1    36253.849384
2    37427.961412
Name: Adjusted notional (thousands), dtype: float64

In [25]:
middle_df['Supervisory delta'] * middle_df['Adjusted notional (thousands)']

0    78693.868057
1   -36253.849384
2   -10082.913813
dtype: float64

In [26]:
hedging_list_generator(middle_df)

[78693.86805747332, -36253.849384403635, -10082.913813053285]

In [31]:
time_bucket_classifier(example_1_df, middle_df)

In [32]:
# EffectiveNotional generator for interest rate
def effectiveNotional(original_df, middle_df, group_name):
    hedging_set = hedging_list_generator(middle_df)
    (row_num, col_num) = original_df.shape
    en_list = [0, 0, 0]
    idx = 0
    for i in range(0,row_num): 
        if (str(middle_df.at[i, 'Hedging set']) == str(group_name)):
            idx = middle_df.at[i, 'Time Bucket'] - 1
            en_list[idx] = hedging_set[i]
            
    #print(en_list)
    e_n = 0
    for item in en_list:
        e_n += item ** 2
    #print(e_n)
    e_n += 1.4 * (en_list[0]) * (en_list[1])
    e_n += 1.4 * (en_list[1]) * (en_list[2])
    e_n += 0.6 * (en_list[0]) * (en_list[2])
    #print(e_n)
    result = math.sqrt(e_n)
    return result

In [33]:
effectiveNotional(example_1_df, middle_df, 'USD')

59269.9634637104

In [34]:
# addOn for Interest Rate
def addOn_interst_rate(original_df, middle_df):
    unique_group = list(set(middle_df['Hedging set'].tolist()))
    effectiveNotionList = []
    for hedging_set in unique_group:
        effectiveNotionList.append(effectiveNotional(original_df, 
                                                       middle_df,
                                                      hedging_set))
    #print(effectiveNotionList)
    newList = 0.005 * np.array(effectiveNotionList)
    #print(newList)
    addOn = np.sum(0.005 * np.array(effectiveNotionList))
    return round(addOn)

In [35]:
time_bucket_classifier(example_1_df, middle_df)

In [36]:
addOn_interst_rate(example_1_df, middle_df)

347

In [37]:
# find the multipler for interest rate EAD
def multiplier_finder(original_df, middle_df):
    addOn = addOn_interst_rate(original_df, middle_df)
    floor = 0.05
    v_c = original_df[['Market value (thousands)']].sum()
    denominator = 2 * (1 - floor) * addOn
    alternative = floor + (1 - floor) * math.exp(v_c / denominator)
    multiplier = min(1, alternative)
    return multiplier

In [38]:
multiplier_finder(example_1_df, middle_df)

1

In [47]:
# EAD for interest rate
def interest_rate_EAD(original_df, middle_df):
    r_c = rc(original_df)
    multiplier = multiplier_finder(original_df, middle_df)
    addon_val = addOn_interst_rate(example_1_df, middle_df)
    return 1.4 * (r_c + multiplier * addon_val)

In [48]:
interest_rate_EAD(example_1_df, middle_df)

569.8